In [1]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder
from pipeline_operations import select
# File system manangement
import os

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

ImportError: cannot import name select

In [2]:
ROOT_DIRECTORY = '../data/home-credit-default-risk'
print(os.listdir(ROOT_DIRECTORY))

['application_test.csv', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [4]:
data['TARGET'].head()

0    1
1    0
2    0
3    0
4    0
Name: TARGET, dtype: int64

In [3]:
columns = ['SK_ID_CURR', 'TARGET']
data[columns].apply(max)

NameError: name 'data' is not defined

In [11]:
pd.Series.value_counts

<unbound method Series.value_counts>

In [8]:
str(['c','v']).replace(" ","")

"['c','v']"

In [12]:
class A:
    def __init__(self, a):
        self.a = a
    def b(self):
        print 'hello'
class B(A):
     def __init__(self):
        self.b()

In [14]:
type([1,2])

list